In [1]:
import networkx as nx
from pybbn.probabilistic import create_reasoning_model

def get_model():
    d = nx.DiGraph()
    d.add_nodes_from(['drug', 'gender', 'recovery'])
    d.add_edges_from([('gender', 'drug'), ('gender', 'recovery'), ('drug', 'recovery')])
    
    p = {
        'gender': {
            'columns': ['gender', '__p__'],
            'data': [
                ['male', 0.51], ['female', 0.49]
            ]
        },
        'drug': {
            'columns': ['gender', 'drug', '__p__'],
            'data': [
                ['female', 'no', 0.24],
                ['female', 'yes', 0.76],
                ['male', 'no', 0.76],
                ['male', 'yes', 0.24]
            ]
        },
        'recovery': {
            'columns': ['gender', 'drug', 'recovery', '__p__'],
            'data': [
                ['female', 'no', 'no', 0.90],
                ['female', 'no', 'yes', 0.10],
                ['female', 'yes', 'no', 0.27],
                ['female', 'yes', 'yes', 0.73],
                ['male', 'no', 'no', 0.99],
                ['male', 'no', 'yes', 0.01],
                ['male', 'yes', 'no', 0.07],
                ['male', 'yes', 'yes', 0.93]
            ]
        }
    }
    
    model = create_reasoning_model(d, p)
    return model

model = get_model()

In [2]:
def o(kv):
    return {k: model.create_observation_evidences(k, v) for k, v in kv.items()}
    
e = o({'gender': 'male', 'drug': 'no'})

f = model.pquery(evidences=e)['recovery'].set_index(['recovery'])['__p__']
a = 1 - f
f, a

(recovery
 no     0.99
 yes    0.01
 Name: __p__, dtype: float64,
 recovery
 no     0.01
 yes    0.99
 Name: __p__, dtype: float64)

In [3]:
e = o({'gender': 'male', 'drug': 'yes'})

c = model.pquery(evidences=e)['recovery'].set_index(['recovery'])['__p__']
p = c + a
c, p

(recovery
 no     0.07
 yes    0.93
 Name: __p__, dtype: float64,
 recovery
 no     0.08
 yes    1.92
 Name: __p__, dtype: float64)

In [4]:
p / p.sum()

recovery
no     0.04
yes    0.96
Name: __p__, dtype: float64